In [38]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from nltk.corpus import stopwords


In [39]:

#import data
stock_df = pd.read_csv('2buysell.csv')
stock_df.head()

,Date,Content,Action
0,1/2/2018,If Procter & Gamble were sponsoring a 2018 adv...,Buy
1,1/2/2018,The golden age of television programming is he...,Buy
2,1/2/2018,Please take your seats.It is that time of year...,Buy
3,1/2/2018,HONG KONG — One of the world’s most valuable s...,Buy
4,1/2/2018,WASHINGTON — United States officials have effe...,Buy


In [40]:
import nltk
nltk.download('stopwords')

#clean up the text
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

#function to clean the text
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fifi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
#apply function to each row in the dataframe
stock_df['Content'] = buysell_df['Content'].apply(clean_text)
#number of words after clean up
print(f"Number of words AFTER clean up: {stock_df['Content'].apply(lambda x: len(x.split(' '))).sum()}") 
stock_df.head()

Number of words AFTER clean up: 2663855


,Date,Content,Action
0,1/2/2018,procter gamble sponsoring 2018 advertisement a...,Buy
1,1/2/2018,golden age television programming heading gris...,Buy
2,1/2/2018,please take seatsit time year annual closing d...,Buy
3,1/2/2018,hong kong one worlds valuable startups got way...,Buy
4,1/2/2018,washington united states officials effectively...,Buy


In [45]:
stock_df['Content'].iloc[0]

'procter gamble sponsoring 2018 advertisement avoiding activists might include inhouse remedy removing corporate stains investors seeking new strategies dividends executive changes board seats arent going away anything siege pg nelson peltz whose trian partners gained directors seat 10month campaign embolden others take cause resistance isnt futile needs carefully consideredfrom general motors whole foods company limits 2017 pg proved size defense 230 billion conglomerate largest ever target proxy fight companies market capitalization 10 billion made 214 percent targets first 11 months 2017 two percentage points previous year according activist insight expect trend continue 2018 even though activists went home emptyhanded cases 11 months wasnt case previous yearfor companies best defense good offense gms mary barra handily fended david einhorns attempt split stock helped already begun address stocks discount selling automakers european brands mr einhorn launched campaign carefully outl

In [42]:
#delete rows with no content

#keep a count of how many you delete
bad_rows = []
for i in range(0,stock_df.shape[0]):
    #if the number of characters is below 100
    if len(stock_df['Content'].iloc[i])<100:
        bad_rows.append(i)

In [35]:
print(stock_df.shape)
print(stock_df.head())


(4574, 3)
       Date                                            Content Action
0  1/2/2018  procter gamble sponsoring 2018 advertisement a...    Buy
1  1/2/2018  golden age television programming heading gris...    Buy
2  1/2/2018  please take seatsit time year annual closing d...    Buy
3  1/2/2018  hong kong one worlds valuable startups got way...    Buy
4  1/2/2018  washington united states officials effectively...    Buy


In [43]:
#Split the data set
X = stock_df.Content
y = stock_df.Action
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [44]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

#convert to a matrix of token counts (CountVectorizer)
#then transform a count matrix to a normalized tf-idf representation (tf-idf transformer)
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.3879781420765027
              precision    recall  f1-score   support

         Buy       0.00      0.00      0.00       293
        Hold       0.39      1.00      0.56       354
        Sell       1.00      0.00      0.01       268

   micro avg       0.39      0.39      0.39       915
   macro avg       0.46      0.33      0.19       915
weighted avg       0.44      0.39      0.22       915



C:\Users\Fifi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
